In [ ]:
import torch
from torch_geometric.data import Data
from torch.nn import Sequential, Linear, ReLU
from torch_geometric.nn import NNConv
from torch_geometric.loader import DataLoader
import numpy as np
# Example graph with 4 edges
# edge_index = torch.tensor([[0, 1, 2, 3], [1, 2, 3, 4]], dtype=torch.long)

# # Edge features (distance, conductivity, etc.)
# edge_features = torch.tensor([[0.1, 1.5], [0.2, 1.0], [0.3, 0.8], [0.4, 1.2]], dtype=torch.float)

# # Node features (3 features per node)
# node_features = torch.rand(5, 3)

# graph_data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_features)

**Running on Colab**
*   Need to pip install torch_geometric
*   Correct the filename as needed.




In [ ]:
import h5py

filename = "./sample_data/Initial_test.h5"
# T_output = []
# edge_distances = []
# edge_mappings = []
# one_hot_features = []
graphs = []

with h5py.File(filename, "r") as f:
    h5_graphs = list(f.keys())
    # for key in f.keys():
    for i in range(len(h5_graphs)):
        print(h5_graphs[i])
        graph = h5_graphs[i]

        T_output = f[graph]["Temperature"][()]
        edge_distances = (f[graph]["edge_distance"][()])
        edge_mappings = f[graph]["edge_mappings"][()] - 1 #adjusting for Python Indexing
        one_hot_features = f[graph]["one_hot_features"][()]

        # print(T_output.shape)
        # print(edge_distances.shape)
        # print(edge_mappings.shape)
        # print(one_hot_features.shape)

        #

        edge_mappings = edge_mappings.astype(np.int64)

        graphs.append(Data(
        x= torch.from_numpy(one_hot_features.reshape(-1,3)).float(),
        edge_index=torch.from_numpy(edge_mappings.reshape(2,-1)),
        edge_attr=torch.from_numpy(edge_distances.reshape(-1,1)).float(),  # 1 feature distance
        y=torch.from_numpy(T_output.reshape(-1,1)).float()  # Target output for each node
        ))

loader = DataLoader(graphs, batch_size=2, shuffle=True)

graph_1
graph_2


In [ ]:
class EdgeFeatureGNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, edge_feature_dim, output_dim):
        super().__init__()

        # Output size of edge_nn must match (hidden_dim * input_dim)
        edge_nn_1 = Sequential(
            Linear(edge_feature_dim, hidden_dim * input_dim),
            # Linear(edge_feature_dim, edge_feature_dim),
            ReLU()
        )

        edge_nn_2 = Sequential(
            Linear(edge_feature_dim, hidden_dim * hidden_dim),
            # Linear(edge_feature_dim, edge_feature_dim),
            ReLU()
        )

        self.conv1 = NNConv(input_dim, hidden_dim, edge_nn_1, aggr='mean')
        self.conv2 = NNConv(hidden_dim, hidden_dim, edge_nn_2, aggr='mean')

        self.fc = Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        # print(x.shape)
        x = self.conv1(x, edge_index, edge_attr).relu()
        # print(x.shape)
        x = self.conv2(x, edge_index, edge_attr).relu()
        return self.fc(x)

# Dummy graph data setup
# node_features = torch.rand(6, 3)  # 6 nodes with 3 features
# edge_index = torch.tensor([[0, 1, 2, 3, 4], [1, 2, 3, 4, 5]], dtype=torch.long)
# edge_features = torch.rand(5, 2)  # 5 edges with 2 features

# graph_data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_features)

# model = EdgeFeatureGNN(input_dim=3, hidden_dim=8, edge_feature_dim=2, output_dim=1)
# output = model(graph_data)
# print("Node-level predictions shape:", output.shape)

In [ ]:
model = EdgeFeatureGNN(input_dim=3, hidden_dim=16, edge_feature_dim=1, output_dim=1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.MSELoss()

# Training loop
epochs = 50
for epoch in range(epochs):
    total_loss = 0
    for batch in loader:
        optimizer.zero_grad()

        # Forward pass
        out = model(batch)
        loss = loss_fn(out, batch.y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {total_loss:.4f}")

Epoch 1, Loss: 37526260.0000
Epoch 2, Loss: 37522704.0000
Epoch 3, Loss: 37519088.0000
Epoch 4, Loss: 37515284.0000
Epoch 5, Loss: 37511248.0000
Epoch 6, Loss: 37506924.0000
Epoch 7, Loss: 37502260.0000
Epoch 8, Loss: 37497192.0000
Epoch 9, Loss: 37491672.0000
Epoch 10, Loss: 37485588.0000
Epoch 11, Loss: 37478864.0000
Epoch 12, Loss: 37471352.0000
Epoch 13, Loss: 37463016.0000
Epoch 14, Loss: 37453768.0000
Epoch 15, Loss: 37443548.0000
Epoch 16, Loss: 37432248.0000
Epoch 17, Loss: 37419788.0000
Epoch 18, Loss: 37406064.0000
Epoch 19, Loss: 37390980.0000
Epoch 20, Loss: 37374360.0000
Epoch 21, Loss: 37356148.0000
Epoch 22, Loss: 37336096.0000
Epoch 23, Loss: 37314048.0000
Epoch 24, Loss: 37289976.0000
Epoch 25, Loss: 37263692.0000
Epoch 26, Loss: 37235088.0000
Epoch 27, Loss: 37204004.0000
Epoch 28, Loss: 37170284.0000
Epoch 29, Loss: 37133764.0000
Epoch 30, Loss: 37093952.0000
Epoch 31, Loss: 37050852.0000
Epoch 32, Loss: 37004312.0000
Epoch 33, Loss: 36954160.0000
Epoch 34, Loss: 369